# Geospatial Data Exploration, Analysis, and Visualization
#### A sample of relevant concepts and methods that can be used for ICESat-2 data analysis*
*Not intended for billions of ATL03 points

ICESat-2 hackweek  
June 12, 2020  
David Shean

You've used the NSIDC API, pulled your granules, extracted the relevant records and fields, and done some initial QA/QC.  Now what?  

Let's work through some Python geospatial data science tools that can help with analysis and visualization of these point data, enabling exploratory data analysis and data-driven discovery.  

# Objectives

* Review fundamental concepts in geospatial analysis (e.g. coordinate systems, projections, datums, geometry types)
* Learn basic geospatial data manipulation and exploration with a relatively small, clean ICESat GLAS dataset
* Use modern data science tools (pandas) for basic data exploration, interpretation, and visualization
* Explore different approaches for spatial aggregation of points - groupby attribute, spatial join by polygon, hex bins
* Learn how to sample a raster at point locations
* Analyze elevation change using sparse laser altimetry data over mountain glaciers

# Tutorial prep

1. Log on to the event Jupyterhub: https://icesat-2.hackweek.io

1. Open a terminal in Jupyterlab ("+" icon in upper left corner, then click on the terminal icon)

1. Navigate to local directory where you want to store tutorial material (home directory by default):  
`cd ~/tutorials`

1. If you haven't dont so, clone the repo:  
`git clone https://github.com/ICESAT-2HackWeek/geospatial-analysis.git`

1. Enter the tutorial directory:  
`cd geospatial-analysis`

    1. If you cloned earlier in the week, you will need to pull latest changes:
`git pull`

**Remeber to post all questions to the `#questions` channel on the Slack workspace**

# Introduction

## Quick Zoom poll:
In the Zoom Participants panel, use the Yes (green) or No (red) response to answer the following question: 

**Have you ever taken a GIS course?**

## GIS Basics
>A geographic information system (GIS) is a framework for gathering, managing, and analyzing data. Rooted in the science of geography, GIS integrates many types of data. It analyzes spatial location and organizes layers of information into visualizations using maps and 3D scenes. ​With this unique capability, GIS reveals deeper insights into data, such as patterns, relationships, and situations—helping users make smarter decisions.

https://www.esri.com/en-us/what-is-gis/overview

Primary data types:
* Vector - points, lines, polygons; shapefiles
    * https://automating-gis-processes.github.io/site/notebooks/L1/geometric-objects.html
* Raster - images, gridded data; GeoTiff

Concepts (with specific geopandas doc):
* Coordinate Systems - map projection and datum: https://geopandas.org/projections.html
* Geometry manipulation/operations: 
    * https://geopandas.org/geometric_manipulations.html
    * https://geopandas.org/set_operations.html

## The Scientific Python landscape

* Python
* Jupyter/iPython
* NumPy, Pandas, Matplotlib, SciPy
* xarray, scikit-learn

One (aging) interpretation of this stack:

![2017 Scientific Python Stack](https://devopedia.org/images/article/60/7938.1587985662.jpg)  
Slide from Jake VanderPlas’s presentation at PyCon 2017, entitled “The Unexpected Effectiveness of Python in Science.”

## The Geospatial Python landscape
* GDAL/OGR, GEOS, PROJ
* rasterio, fiona, shapely, pyproj
* geopandas, cartopy, xarray
* rioxarray/salem, PDAL

[Insert shiny new figure here]

## Many excellent resources available
Here are a few...

#### Full courses:
* https://www.earthdatascience.org/courses/
* https://automating-gis-processes.github.io
* https://github.com/UW-GDA/gda_course_2020

#### Geohackweek: 
* https://geohackweek.github.io/

#### Pangeo:
* https://github.com/pangeo-data/pangeo-tutorial

## Complementary approaches for ICESat-2 data

1. Efficient, scalable processing of huge point datasets
    * Mostly NumPy
    * Basic array manipulations
    * Array size limited by memory - can chunk with Dask
2. Higher-level data science - *analysis, interpetation, and visualization*
    * NumPy under the hood
    * Convenience and flexibility comes with small performance hit
    * Labels make life easier (don't need to remember integer indices)

Here, we're going to explore #2 with an existing dataset.

At the end of the day, most applications just need clean (x,y,z,t) points.

As with all things in the *nix/open-source/Python world, there are always multiple approaches that can be used to accomplish the same goals.  The user must decide on an approach based on complexity, time constraints, etc.

# ICESat GLAS Background
The NASA Ice Cloud and land Elevation Satellite ([ICESat](https://icesat.gsfc.nasa.gov/icesat/)) was a NASA mission carrying the Geosciences Laser Altimeter System (GLAS) instrument: a space laser, pointed down at the Earth (and unsuspecting Earthlings).  

It measured surface elevations by precisely tracking laser pulses emitted from the spacecraft at a rate of 40 Hz (a new pulse every 0.025 seconds).  These pulses traveled through the atmosphere, reflected off the surface, back up through the atmosphere, and into space, where some small fraction of that original energy was received by a telescope on the spacecraft.  The instrument electronics precisely recorded the time when these intrepid photons left the instrument and when they returned.  The position and orientation of the spacecraft was precisely known, so the two-way traveltime (and assumptions about the speed of light and propagation through the atmosphere) allowed for precise forward determination of the spot on the Earth's surface (or cloud tops, as was often the case) where the reflection occurred.  The laser spot size varied during the mission, but was ~70 m in diameter. 

ICESat collected billions of measurements from 2003 to 2009, and was operating in a "repeat-track" mode that sacrificed spatial coverage for more observations along the same ground tracks over time.  One primary science focus involved elevation change over the Earth's ice sheets.  It allowed for early measurements of full Antarctic and Greenland ice sheet elevation change, which offered a detailed look at spatial distribution and rates of mass loss, and total ice sheet contributions to sea level rise.  

There were problems with the lasers during the mission, so it operated in short campaigns lasting only a few months to prolong the full mission lifetime.  While the primary measurements focused on the polar regions, many measurements were also collected over lower latitudes, to meet other important science objectives (e.g., estimating biomass in the Earth's forests, observing sea surface height/thickness over time). 

# Part 1: Sample GLAS dataset for CONUS
A few years ago, I wanted to evaluate ICESat coverage of the Continental United States (CONUS).  The primary application was to extract a set of accurate control points to co-register a large set of high-resolution digital elevation modoels (DEMs) derived from satellite stereo imagery.  I wrote some Python/shell scripts to download, filter, and process all of the GLAH14 granules in parallel ([https://github.com/dshean/icesat_tools](https://github.com/dshean/icesat_tools)).

The high-level workflow is here: https://github.com/dshean/icesat_tools/blob/master/glas_proc.py#L24.  These tools processed each HDF5 (H5) file and wrote out csv files containing only the “good” points. These csv files were concatenated to prepare the single input csv (`GLAH14_tllz_conus_lulcfilt_demfilt.csv`) that we will use for this tutorial.  

The csv contains ICESat GLAS shots that passed the following filters:
* Within some buffer (~110 km) of mapped glacier polygons from the [Randolph Glacier Inventory (RGI)](https://www.glims.org/RGI/)
* Returns from exposed bare ground (landcover class 31) or snow/ice (12) according to a 30-m Land-use/Land-cover dataset (2011 NLCD, https://www.mrlc.gov/data?f%5B0%5D=category%3Aland%20cover)
* Elevation values within some threshold (200 m) of elevations sampled from an external reference DEM (void-filled 1/3-arcsec [30-m] SRTM-GL1, https://lpdaac.usgs.gov/products/srtmgl1v003/), used to remove spurious points and returns from clouds.
* Various other ICESat-specific quality flags (see comments in `glas_proc.py` for details)

The final file contains a relatively small subset (~65K) of the total shots in the original GLAH14 data granules from the full mission timeline (2003-2009).  The remaining points should represent returns from the Earth's surface with reasonably high quality, and can be used for subsequent analysis.

## Wait, I thought this was an ICESat-**2** hackweek?

Note that we could (and should!) swap similarly processed/filtered ATL06 points over CONUS for this tutorial.  I did not have time to do this before the hackweek, but it would make a great project (nudge).  After opening this updated file with Pandas, it should mostly be a matter of updating field names throughout the notebook.

# Import necessary modules

In [ ]:
import os
import requests
import zipfile

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Magic function to enable interactive plotting (zoom/pan) in Jupyter notebook
#If running locally, this would be `%matplotlib notebook`, but since we're using Juptyerlab, we use widget
%matplotlib widget
#%matplotlib inline

In [ ]:
#Define path to sample GLAS data
glas_fn = 'GLAH14_tllz_conus_lulcfilt_demfilt.csv'

In [ ]:
#Quick check of csv file contents
!head $glas_fn

# Pandas

Trust me, you should learn how to use `Pandas`, regardless of your ICESat-2 application.  

A significant portion of the Python data science ecosystem is based on Pandas and/or Pandas data models.

>pandas is a Python package providing fast, flexible, and expressive data structures designed to make working with "relational" or "labeled" data both easy and intuitive. It aims to be the fundamental high-level building block for doing practical, real world data analysis in Python. Additionally, it has the broader goal of becoming the most powerful and flexible open source data analysis / manipulation tool available in any language. It is already well on its way towards this goal.

https://github.com/pandas-dev/pandas#main-features

If you are working with tabular data (rows and columns, like a csv or spreadsheet), especially time series data, please use pandas.
* A better way to deal with tabular data, built on top of NumPy arrays
* With NumPy, we had to remember which column number (e.g., 3, 4) represented each variable (lat, lon, glas_z, etc)
* Pandas allows you to store data with different types, and then reference using more meaningful labels
    * NumPy: `glas_np[:,4]`
    * Pandas: `glas_df['glas_z']`
* A good "10-minute" reference with examples: https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html

If you are working with more complex data, like collections of tabular time series data from 100s of met stations or netCDF model output, you can use [`xarray` package](http://xarray.pydata.org/en/stable/), which extends the `pandas` data model to n-dimensions.

## Load the csv file with Pandas
* Note that pandas has excellent readers for most common file formats: https://pandas.pydata.org/pandas-docs/stable/reference/io.html

In [ ]:
glas_df = pd.read_csv(glas_fn)

## That was easy.  Let's inspect the returned `DataFrame` object

In [ ]:
glas_df

## Check data types
* Can use the DataFrame `info` method

In [ ]:
glas_df.info()

## Get the column labels
* Can use the DataFrame `columns` attribute

In [ ]:
glas_df.columns

If you are new to Python and object-oriented programming, take a moment during the break to consider the [difference between the methods and attributes](https://stackoverflow.com/questions/46312470/difference-between-methods-and-attributes-in-python) of the DataFrame, and how both are accessed. 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html

If this is confusing, ask your neighbor or instructor.

## Preview records using DataFrame `head` and `tail` methods

In [ ]:
glas_df.head()

In [ ]:
glas_df.tail()

## Compute the mean and standard deviation for all values in each column

In [ ]:
glas_df.mean()

In [ ]:
glas_df.std()

## Apply a custom function to each column
* For this example, let's define a function to compute the Normalized Median Absolute Deviation (NMAD)
* https://en.wikipedia.org/wiki/Median_absolute_deviation
    * For a normal distribution, this is equivalent to the standard deviation. 
    * For data containing outliers, it is a more robust representation of variability. 
* We will then use the Pandas `apply` method to compute the NMAD for all values in each column

In [ ]:
def nmad(a, c=1.4826):
    return np.median(np.fabs(a - np.median(a))) * c

In [ ]:
glas_df.apply(nmad)

*Note: the NMAD function is now distributed with `scipy.stats`: 
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.median_absolute_deviation.html*

Can also use and `apply` functions imported from modules

In [ ]:
import scipy.stats

In [ ]:
glas_df.apply(scipy.stats.median_absolute_deviation)

## Print quick stats for entire DataFrame with the `describe` method

In [ ]:
glas_df.describe()

Useful, huh?  Note that the `50%` statistic is the median.

## Use the Pandas plotting functionality to create a 2D scatterplot of `glas_z` values
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.scatter.html
* Note that labels and colorbar are automatically plotted!
* Can adjust the size of the points using the `s=1` keyword
* Can experiment with different color ramps:
    * https://matplotlib.org/examples/color/colormaps_reference.html (I prefer `inferno`)

#### Color ramps
Information on how to choose a good colormap for your data: https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html  
Another great resource (Thanks @fperez!): https://matplotlib.org/cmocean/  
**TL;DR** Don't use `jet`, use a perceptually uniform colormap for linear variables like elevation. Use a diverging color ramp for values where sign is important.

In [ ]:
glas_df.plot(x='lon', y='lat', kind='scatter', c='glas_z', s=1, cmap='inferno');

## Experiment by changing the variable represented with the color ramp
* Try `decyear` or other columns to quickly visualize spatial distribution of these values.

In [ ]:
glas_df.plot(x='lon', y='lat', kind='scatter', c='decyear', s=1, cmap='inferno');

## Create a histogram that shows the number of points vs time (`decyear`)

In [ ]:
#Determine number of bins needed to provide weekly resolution
nbins = int(50 * np.ptp(glas_df['decyear'].values))
nbins

In [ ]:
ax = glas_df.hist('decyear', bins=nbins)[0,0]
ax.set_ylabel('Number of ICESat points')
ax.set_title('Weekly usable ICESat point count over CONUS');

Note that we can resolve the distinct campaigns during the ICESat mission (each ~1-2 months long).

## Create a histogram of all `glas_z` elevation values

In [ ]:
#Determine number of bins needed for 20 m elevation bins
binwidth = 20
nbins = int(np.ptp(glas_df['glas_z'].values)/binwidth)
nbins

In [ ]:
glas_df.hist('glas_z', bins=nbins);

## Wait a minute...negative elevations!?  Who calibrated this thing? C'mon NASA.

## A note on vertical datums

Note that some elevations are less than 0 m.  How can this be?

The `glas_z` values are height above (or below) the WGS84 ellipsoid. This is not the same vertical datum as mean sea level (roughly approximated by a geoid model).

A good resource explaining the details: https://vdatum.noaa.gov/docs/datums.html

## How many GLAS points have a negative `glas_z` value?

In [ ]:
glas_df['glas_z'] < 0

In [ ]:
(glas_df['glas_z'] < 0).value_counts()

## Check spatial distribution of points below 0 (height above WGS84 ellipsoid)
* Create a scatterplot only using points with negative values
* Adjust the color ramp bounds to bring out more detail for these points
    * hint: see the `vmin` and `vmax` arguments for the `plot` function
* What do you notice about these points? (may be tough without more context, like coastlines and state boundaries or a tiled basemap - we'll learn how to incorporate these later)

In [ ]:
glas_df[glas_df['glas_z'] < 0]

In [ ]:
glas_df[glas_df['glas_z'] < 0].plot(x='lon', y='lat', kind='scatter', c='dem_z', s=1, cmap='inferno', vmin=-30, vmax=0);

## Geoid offset grids
PROJ and GDAL use raster grids of the horizontal and vertical offsets between the WGS84 ellipsoid and different geoid models.

These grids are now hosted on the cloud: https://cdn.proj.org/, which enables on-the-fly transformations with GDAL!

Here's the EGM96 offset grid.  It shows the height differene between the WGS84 ellipsoid (simple shape model of the Earth) and the EGM96 geoid, which approximates a geopotential (gravitational) surface, approximately euivalent to mean sea level 

![EGM96 geoid offset grid](https://raw.githubusercontent.com/UW-GDA/gda_course_2020/master/resources/sample_img/egm96_offset.png)  
(Source GeoTiff: https://cdn.proj.org/us_nga_egm96_15.tif)


Note values for CONUS.  

A lot of the points with elevation < 0 m in the above plot are near coastal sites, roughly near mean sea level.  We see that the geoid offset (difference between WGS84 ellipsoid and EGM96 geoid in this case) for CONUS is roughly -20 m.  So the ICESat GLAS point elevations near the coast are roughly -20 m relative to the ellipsoid, even though they are 0 m relative to the geoid (approximately mean sea level).  These concepts around datums can be a bit confusing, so please ask questions.

Note that ICESat-2 includes elevation values relative to the EGM2008 geoid model to provide orthometric heights - a more recent, more accurate, more detailed geoid model compared to EGM96.

# Part 2: More Pandas, Outlier Removal, Groupby

## Compute the elevation difference between ICESat `glas_z` and SRTM `dem_z` values

Earlier, I mentioned that I had sampled the SRTM DEM for each GLAS point.  See Appendix for an example of how to do this with rasterio.  

For now, let's use the previously sampled values, compute an elevation difference and store in a new column in our DataFrame called `glas_srtm_dh`

Remember the order of this calculation (if the difference values are negative, which dataset is higher elevation?)

In [ ]:
glas_df['glas_srtm_dh'] = glas_df['glas_z'] - glas_df['dem_z']
glas_df.head()

## Compute the time difference between ICESat point timestamp and the SRTM timestamp
* Store in a new column named `glas_srtm_dt`
* The SRTM data were collected between February 11-22, 2000
    * Can assume a constant decimal year value of 2000.112 for now

In [ ]:
#February 11-22, 2000
srtm_decyear = 2000.112
glas_df['glas_srtm_dt'] = glas_df['decyear'] - srtm_decyear
glas_df.head()

## Compute *apparent* annualized elevation change rate ($\frac{dh}{dt}$ in meters per year) from these new columns
* This will be rate of change between the SRTM timestamp (2000) and each GLAS point timestamp (2003-2009)

In [ ]:
glas_df['glas_srtm_dhdt'] = glas_df['glas_srtm_dh']/glas_df['glas_srtm_dt']
glas_df.head()

## Create a scatterplot of the difference values
* Use a symmetrical `RdBu` (Red to Blue) color ramp
* Set the color ramp limits using `vmin` and `vmax` keyword arguments to be symmetrical about 0 z

In [ ]:
ax = glas_df.plot(x='lon', y='lat', kind='scatter', c='glas_srtm_dh', s=1, cmap='RdBu', vmin=-10, vmax=10)

In [ ]:
ax = glas_df.plot(x='lon', y='lat', kind='scatter', c='glas_srtm_dh', s=1, cmap='RdBu', vmin=-50, vmax=50)

## Compute some descriptive statistics
* Why might we have a non-zero mean/median difference?

In [ ]:
print(glas_df['glas_srtm_dh'].mean())
print(glas_df['glas_srtm_dh'].std())

In [ ]:
print(glas_df['glas_srtm_dh'].median())
print(nmad(glas_df['glas_srtm_dh']))

## Create a histogram of the difference values

In [ ]:
f, ax = plt.subplots()
glas_df.hist('glas_srtm_dh', ax=ax, bins=128, range=(-10,10));
ax.axvline(0, color='k')
ax.axvline(glas_df['glas_srtm_dh'].median(), color='r');

## Create a scatterplot of elevation difference `glas_srtm_dh` values vs elevation values
* `glas_srtm_dh` should be on the y-axis
* `glas_z` values on the x-axis

In [ ]:
ax = glas_df.plot('glas_z', 'glas_srtm_dh', kind='scatter', s=1)
#Add a horizontal line at 0
ax.axhline(0, color='k', lw=0.5);

## Remove outliers
The initial filter in `glas_proc.py` removed GLAS points with absolute elevation difference >200 m compared to the SRTM elevations.  We expect most real elevation change signals to be less than this for the given time period.  But clearly some outliers remain.

Let's design and apply a simple filter that removes outliers.  One option is to define outliers as values outside some absolute threshold. Can set this threshold as some multiple of the standard deviation (e.g., `3*std`). Can also use quantile or percentile values for this.

In [ ]:
print("Mean difference:", glas_df['glas_srtm_dh'].mean())
thresh = 3.5 * glas_df['glas_srtm_dh'].std()
print("3.5 * std:", thresh)

In [ ]:
idx = (glas_df['glas_srtm_dh'] - glas_df['glas_srtm_dh'].mean()).abs() <= thresh
glas_df_fltr = glas_df[idx]
print("Number of points before filter:", glas_df.shape[0])
print("Number of points after filter:", glas_df_fltr.shape[0])

In [ ]:
clim = thresh
f, axa = plt.subplots(1,2, figsize=(10,4))
#Outliers plotted in black
glas_df.plot(ax=axa[1], x='glas_z', y='glas_srtm_dh', kind='scatter', s=1, color='k', label='Outliers')
glas_df_fltr.plot(ax=axa[0], x='lon', y='lat', kind='scatter', c='glas_srtm_dh', s=1, cmap='RdBu', vmin=-clim, vmax=clim)
glas_df_fltr.plot(ax=axa[1], x='glas_z', y='glas_srtm_dh', kind='scatter', s=1, c='orange', label='Inliers')
glas_df[~idx].plot(ax=axa[0], x='lon', y='lat', kind='scatter', color='k', s=1, legend=False)
axa[1].axhline(0,color='k')
plt.tight_layout()

## Active remote sensing sanity check

Even after removing outliers, there are still some big differences between the SRTM and GLAS elevation values.  

* Do you see systematic differences between the glas_z and dem_z values?
* Any clues from the scatterplot? (e.g., do some tracks (north-south lines of points) display systematic bias?)
* Brainstorm some ideas about what might be going on here.  Think about the nature of each sensor:
    * ICESat was a Near-IR laser (1064 nm wavelength) with a big ground spot size (~70 m in diameter)
        * Timestamps span different seasons between 2003-2009
    * SRTM was a C-band radar (5.3 GHz, 5.6 cm wavelength) with approximately 30 m ground sample distance (pixel size)
        * Timestamp was February 2000
        * Data gaps (e.g., radar shadows, steep slopes) were filled with ASTER GDEM2 composite, which blends DEMs acquired over many years ~2000-2014
* Consider different surfaces and how the laser/radar footprint might be affected:
    * Flat bedrock surface
    * Dry sand dunes
    * Steep montain topography like the Front Range in Colorado  
    * Dense vegetation of the Hoh Rainforest in Olympic National Park

## Let's check to see if differences are due to our land-use/land-cover (LULC) classes
* Determine the unique values in the `lulc` column (hint: see the `value_counts` method)
* In the introduction, I mentioned that I initially preserved only two classes for these GLAS points (12 - snow/ice, 31 - barren land), so this isn't going to help us over forests:
    * https://www.mrlc.gov/data/legends/national-land-cover-database-2011-nlcd2011-legend

In [ ]:
glas_df['lulc'].value_counts()

## Use Pandas `groupby` to compute stats for the LULC classes
* https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
* This is one of the most powerful features in Pandas, efficient grouping and analysis based on attributes
* Compute mean, median and std of the difference values (glas_z - dem_z) for each LULC class
* Do you see a difference between values over glaciers vs bare rock?

In [ ]:
glas_df[['glas_srtm_dh', 'lulc']].groupby('lulc').mean()

In [ ]:
#Statistics for full dataset
glas_df[['glas_srtm_dh', 'lulc']].groupby('lulc').agg(['mean', 'std', 'median', nmad])

In [ ]:
#Statistics for filtered dataframe (after removing outliers)
glas_df_fltr[['glas_srtm_dh', 'lulc']].groupby('lulc').agg(['mean', 'std', 'median', nmad])

In [ ]:
for title, group in glas_df_fltr.groupby('lulc'):
    group.plot(x='lon', y='lat', kind='scatter', c='glas_srtm_dh', s=1, cmap='RdBu', vmin=-10, vmax=10, title='LULC %i' % title)

# Part 3: Geopandas Intro

pandas is great, but what if we want to do some geospatial operations - like reproject our points or compute the intersection between Point and Polygon features?

Enter Geopandas - all the great things about pandas, plus geo! (http://geopandas.org/).

>"GeoPandas is an open source project to make working with geospatial data in python easier. GeoPandas extends the datatypes used by pandas to allow spatial operations on geometric types. Geometric operations are performed by shapely. Geopandas further depends on fiona for file access and descartes and matplotlib for plotting."

>"GeoPandas enables you to easily do operations in python that would otherwise require a spatial database such as PostGIS."

Under the hood, GeoPandas is `pandas` plus some other core geospatial packages:
* `shapely` for geometry operations (https://shapely.readthedocs.io/en/stable/manual.html)
* `fiona` for reading/writing GIS file formats (https://fiona.readthedocs.io/en/latest/manual.html)
* `pyproj` for projections and coordinate system transformations (http://pyproj4.github.io/pyproj/stable/)

Under those hoods are lower-level geospatial libraries (GEOS, GDAL/OGR, PROJ4) that provide a foundation for most GIS software (open-source and commercial).  I encourage you to explore these - I guarantee you will learn something valuable.

For now, let's explore some basic geopandas functionality.

## Convert pandas `DataFrame` to geopandas `GeoDataFrame`
To do this, we need to create a new column containing standardized `geometry` objects (e.g., `Point`, `Polygon`) for each record in the DataFrame. 

https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html

## Convert the Pandas `DataFrame` to a GeoPandas `GeoDataFrame`
* https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html
* Careful about lon and lat order!
* Define coordinate reference system (4326 is geographic lat/lon on WGS84 Ellispoid) 
    * https://www.earthdatascience.org/courses/earth-analytics/spatial-data-r/intro-to-coordinate-reference-systems/

In [ ]:
glas_gdf = gpd.GeoDataFrame(glas_df, geometry=gpd.points_from_xy(glas_df['lon'], glas_df['lat']), crs='EPSG:4326')

In [ ]:
type(glas_gdf)

In [ ]:
type(glas_df)

In [ ]:
#Same `head()` method works on a GeoDataFrame! inherited from Pandas DataFrame
glas_gdf.head()

In [ ]:
#But we have additional attributes/methods in a GeoDataFrame
glas_gdf.crs

## Create a quick 2D scatterplot

Like a Pandas DataFrame, a GeoDataFrame has convenience plotting function that is built on matlplotlib

In [ ]:
glas_gdf.plot();

OK, looks like a scatterplot.  But let's plot the elevation values with a color ramp.  
To do this, just specify the column name as the first argument to `plot`:

In [ ]:
glas_gdf.plot('glas_z', markersize=1, cmap='inferno', legend=True);

## Load and plot state polygons for context


Hmmm, let's see.  Two choices:
1. We could go to ESRI or the U.S. Census website, identify and download a shapefile, unzip 4+ files, copy/paste the appropriate \*.shp filename into the notebook.  Wait, how can I download on a remote server?  OK, maybe run something like `wget http://...`, unzip, provide absolute path  
*- OR -*
2. Give geopandas a url that points to a GeoJSON file somewhere on the web, and read dynamically

Yeah, let's go with #2

Let's use the US States 5M GeoJSON here: http://eric.clst.org/tech/usgeojson/

We've heard GeoJSON mentioned a few times this week.  It's a great format.  If you are unfamiliar: https://en.wikipedia.org/wiki/GeoJSON

In [ ]:
#1:5000000 scale polygons
states_url = 'http://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_5m.json'
#1:500000 scale polygons (larger file, more vertices)
#states_url = 'http://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_500k.json'

In [ ]:
states_gdf = gpd.read_file(states_url)

### Inspect State GeoDataFrame
Note that some geometry entries are tuples of POLYGON objects - these are states with islands or rings

In [ ]:
states_gdf.head()

In [ ]:
states_gdf.crs

### Limit to Lower48

In [ ]:
idx = states_gdf['NAME'].isin(['Alaska','Puerto Rico','Hawaii'])
states_gdf = states_gdf[~idx]

In [ ]:
ax = glas_gdf.plot('glas_z', markersize=1, cmap='inferno', legend=True)
states_gdf.plot(ax=ax, facecolor='none', edgecolor='0.5', linewidth=0.5);

# RGI glacier polygons

Let's grab some glacier outline poygons from the Randolph Glacier Inventory (RGI) v6.0: https://www.glims.org/RGI/

## Quick Zoom poll:
In the Zoom Participants panel, use the Yes (green) or No (red) response to answer the following question: 

**Have you ever worked with RGI polygons for glacier and/or ice caps?**

In [ ]:
#Fetch the zip file for Region 02 (Western North America)
rgi_zip_fn = '02_rgi60_WesternCanadaUS.zip'

if not os.path.exists(rgi_zip_fn):
    url = 'https://www.glims.org/RGI/rgi60_files/' + rgi_zip_fn
    myfile = requests.get(url)
    open(rgi_zip_fn, 'wb').write(myfile.content)

In [ ]:
#Unzip the file
with zipfile.ZipFile(rgi_zip_fn, 'r') as zip_ref:
    zip_ref.extractall('rgi')

In [ ]:
#Specify the shapefile filename
rgi_fn = 'rgi/02_rgi60_WesternCanadaUS.shp'

## Load RGI shapefile using Geopandas
* Very easy with `read_file()` meethod

In [ ]:
rgi_gdf = gpd.read_file(rgi_fn)
rgi_gdf

That's it!

In [ ]:
#By default a new integer index is created.  Can use the RGI ID as our index
#rgi_gdf = rgi_gdf.set_index('RGIId')

## Create a quick plot

In [ ]:
rgi_gdf.plot();

### Update our quick plot of RGI polygons

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ax = world.plot(alpha=0.2)
states_gdf.plot(ax=ax, facecolor='none', edgecolor='0.5', linewidth=0.5)
rgi_gdf.plot(ax=ax, edgecolor='k', linewidth=0.5);

## Clip RGI polygons to CONUS
GeoPandas makes spatial selection easy.  

We'll have two options: 1) using a bounding box, and 2) using an arbitrary polygon.

### 1. Bounding box

In [ ]:
glas_gdf.total_bounds

#### Define min/max variables for each dimension

In [ ]:
xmin, ymin, xmax, ymax = glas_gdf.total_bounds

#### Create new GeoDataFrame from output of simple spatial filter with GeoPandas `cx` function
* https://geopandas.org/indexing.html

In [ ]:
rgi_gdf_conus = rgi_gdf.cx[xmin:xmax, ymin:ymax]

#### Quick plot to verify

In [ ]:
rgi_gdf_conus.plot(edgecolor='k', linewidth=0.5);

### 2. Clip points to arbitrary Polygon geometry

Let's define a Polygon around our area of interest (the GLAS points) 

To do this, we'll first take the union of our ~65K GLAS points, and then compute the convex hull

This will return a Polygon geometry object, which renders nicely in the Jupyter notebook

In [ ]:
glas_gdf_chull = glas_gdf.unary_union.convex_hull

In [ ]:
#Check the type
type(glas_gdf_chull)

#### Preview geometry
Note that geometry objects (points, lines, polygons, etc.) will render directly in the Jupyter notebook!  Great for quick previews.

In [ ]:
glas_gdf_chull

In [ ]:
print(glas_gdf_chull)

#### Compute intersection between all RGI polygons and the convex hull
Use the GeoDataFrame `intersects()` function.  
This will return a Boolean DataSeries, True if points intersect the polygon, False if they do not

In [ ]:
rgi_gdf_idx = rgi_gdf.intersects(glas_gdf_chull)

In [ ]:
rgi_gdf_idx

#### Extract records with True for the intersection

In [ ]:
print("Number of RGI polygons before:",rgi_gdf.shape[0])
rgi_gdf_conus = rgi_gdf[rgi_gdf_idx]
print("Number of RGI polygons after:", rgi_gdf_conus.shape[0])

#### Quick plot to verify
Note latitude range

In [ ]:
rgi_gdf_conus.plot(edgecolor='k', linewidth=0.5);

# Part 4: Reprojection and Coordinate Systems

All of the above examples used standard geodetic lat/lon coordinate system (EPSG:4326).  This is fine for global analyses and basic visualization.  But remember that the width of a degree of longitude varies with latitude (~111 km near equator, ~0 m near pole), so our plots have scaling and aspect ratio issues.

We need to choose a map projection that is appropriate for our data. This decision is important for visualization, but is also critical for precise quantitative analysis. For example, if you want to compute area or volume change, you should use an equal-area projection. If you want to calculate distances between two points, you should use an equidistant projection.

https://www.axismaps.com/guide/general/map-projections/

Sadly, there is no "perfect" projection. You, as the mapmaker or data analyst, are responsible for choosing a projection with the right characteristics for your purposes. Let's explore a bit further, and we'll revisit some general guidelines later.

## Use GeoPandas to reproject your GeoDataFrame
* Use the very convenient `to_crs()` method to reproject: https://geopandas.org/projections.html
* Start by reprojecting the points to a Universal Transverse Mercator (UTM), Zone 11N (EPSG:32611)
* Store the output as a new GeoDataFrame
* Do a quick `head()` and note the new values in the `geometry` column

In [ ]:
utm_crs = 'EPSG:32611'
glas_gdf_utm = glas_gdf.to_crs(utm_crs)

In [ ]:
glas_gdf_utm.crs

In [ ]:
glas_gdf_utm.head()

## Create a new plot of the reprojected points
* Note the new coordinate system origin (0,0), units, and aspect ratio

In [ ]:
ax = glas_gdf_utm.plot(column='glas_z', cmap='inferno', markersize=1, legend=True)

## Excellent, but what did we just do?

Under the hood, GeoPandas used the `pyproj` library (a Python API for PROJ) to transform each point from one coordinate system to another coordinate system.  

I guarantee that you've all done this kind of thing before, you may just not remember it or recognize it in this context. See: https://en.wikipedia.org/wiki/List_of_common_coordinate_transformations

In 2D, transforming (x,y) coordinates between different projections (e.g., WGS84 vs. UTM) on the same reference ellipsoid is pretty straightforward.  Things start to get more complicated when you include different ellipsoid models, horizontal/vertical datums, etc.  Oh, also the Earth's surface is not static - plate tectonics make everything more complicated, as time becomes important, and transformations must include a "kinematic" component.  

Fortunately, the `PROJ` library (https://proj.org/about.html) has generalized much of the complicated math for geodetic coordinate transformations.  It's been under development since the 1980s, and our understanding of the Earth's shape and plate motions has changed dramatically in that time period.  So, still pretty complicated, and there are different levels of support/sophistication in the tools/libraries that use `PROJ`.

We aren't going to get into the details here, but feel free to take a look at the Transformations section here to get a sense of how this is actually accomplished: https://proj.org/operations/index.html

## Let's define a custom projection for the Western U.S.

The UTM projection we used above is not the best choice for our data, which actually span 4 UTM zones:
https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system#/media/File:Utm-zones-USA.svg. 

We used Zone 11N, which means that our map will have limited distortion within that zone centered on -110°W.  But distortion will increase beyond the width of the -114° to -108°W zone definition.

Let's instead use a custom Albers Equal Area (AEA) projection to minimize distoration over the full spatial extent of our GLAS points

To do this, we'll define a PROJ string (https://proj.org/usage/quickstart.html?highlight=definition), which can be interpreted by most Python geopackages (like `pyproj`).

The Albers Equal Area projection requires two standard parallels: https://proj.org/operations/projections/aea.html.  Here, we will also specify the center latitude and center longitude for the coordinate system origin.

* Define a custom Albers Equal Area proj string `'+proj=aea...'`
    * https://en.wikipedia.org/wiki/Albers_projection
    * PROJ reference, with example: https://proj.org/operations/projections/aea.html
    * Here is another sample proj string example for an Albers Equal Area projection of the U.S. (note that this uses GRS80 ellipsoid and NAD83 datum): http://spatialreference.org/ref/esri/102003/proj4/
        * For your string, please use WGS84 ellipsoid (see the proj doc for aea in the above link)
* Use the center longitude and center latitude you calculated earlier
* Define the two standard parallels (lines of latitude) based on the range of your points
    * Scale is true along these parallels, and distortion increases as you move away from these two parallels
    * One approach would be to use min and max latitude from the `total_bounds` extent computed earlier
        * This is fine, but note that this could lead to additional distortion near your center latitude
        * Extra Credit: figure out how to place them slightly inside your min and max latitude to minimize distortion across the entire latitude range
* Use Python string formatting to dynamically create your proj string (don't just hardcode your values, but substitute variables in the string)
* Print the final proj string

## Get bounding box (extent) and center (lon, lat) of GLAS points
* See GeoPandas API reference. In this case, you want the total_bounds attribute: http://geopandas.org/reference.html#geopandas.GeoSeries.total_bounds
* Center can be calculated from the min/max extent values in each dimension

In [ ]:
glas_extent = glas_gdf.total_bounds
glas_extent

In [ ]:
glas_center = glas_extent.reshape((2,2)).mean(axis=0)
glas_center

## Create the PROJ string

In [ ]:
clon = glas_center[0]
clat = glas_center[1]
#Standard parallels at min/max values
#p1 = glas_extent[1]
#p2 = glas_extent[3]
#Standard parallels within the point extent
dp = (glas_extent[3] - glas_extent[1])*0.67
p1 = clat - dp/2.
p2 = clat + dp/2.

In [ ]:
proj_str_aea = '+proj=aea +lat_1={:.2f} +lat_2={:.2f} +lat_0={:.2f} +lon_0={:.2f}'.format(p1, p2, clat, clon)
proj_str_aea

## Reproject the GLAS points to the custom projection

In [ ]:
glas_gdf_aea = glas_gdf.to_crs(proj_str_aea)

In [ ]:
glas_gdf_aea.head()

In [ ]:
bbox = glas_gdf_aea.total_bounds
bbox

## Create scatter plots for each of the three projections (WGS84, UTM, and custom AEA)

In [ ]:
f, axa = plt.subplots(1,3, figsize=(10,2.5))
glas_gdf.plot(ax=axa[0], column='glas_z', cmap='inferno', markersize=1, legend=True)
axa[0].set_title('WGS84')
axa[0].grid()
glas_gdf_utm.plot(ax=axa[1], column='glas_z', cmap='inferno', markersize=1, legend=True)
axa[1].set_title(utm_crs)
axa[1].grid()
glas_gdf_aea.plot(ax=axa[2], column='glas_z', cmap='inferno', markersize=1, legend=True)
axa[2].set_title('Albers Equal-area')
axa[2].grid()
plt.tight_layout()

## Umm, they kind of look the same.  Why are we wasting time on this?

* Note the location of the origin for each coordinate system
    * The (0,0) should be near the center of your points for the AEA projection
    * Where's the origin for the UTM projection?
* Note how azimuth and distances are distorted around edges of the plots

# Part 5: Spatial Aggregation

## Hexbin plots
Hexbin plots are nice option to visualize the spatial distribution of binned point density or other metric (e.g., median elevation) on a regular grid.  Later in the week, we'll learn about other interpolation and gridding options.

Hexagons are preferable over a standard square/rectangular grid: https://pro.arcgis.com/en/pro-app/tool-reference/spatial-statistics/h-whyhexagons.htm

Also, see Sarah Battersby's publication on the topic (and appreciate one of the better journal article titles I've seen): https://doi.org/10.1080/15230406.2016.1180263

Here are some resources on generating hexbins using Python and matplotlib:
* https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hexbin.html
* http://darribas.org/gds15/content/labs/lab_09.html

Note: an equal-area projection is also a good idea for a hexbin plot.

In [ ]:
nbins = 40

In [ ]:
#To calculate number of bins dynamically with specified bin spacing in meters
#bin_width = 27000 #meters
#nbins_x = int(np.ceil(np.abs(bbox[2] - bbox[0])/bin_width))
#nbins_y = int(np.ceil(np.abs(bbox[3] - bbox[1])/bin_width))
#print(nbins_x, nbins_y)
#nbins = nbins_x

In [ ]:
f,ax = plt.subplots(figsize=(8,6))
hb = ax.hexbin(glas_gdf_aea.geometry.x, glas_gdf_aea.geometry.y, gridsize=nbins, cmap='inferno', bins='log', alpha=0.6, mincnt=1)
plt.colorbar(hb, label='Point Count')
states_gdf.to_crs(proj_str_aea).plot(ax=ax, facecolor='none', edgecolor='black');
#Limit plot extent to GLAS point bounds (note: ax.autoscale(False) doesn't work here)
ax.set_xlim(bbox[[0,2]]);
ax.set_ylim(bbox[[1,3]]);
ax.set_title('GLAS Bin Point Count');

In [ ]:
f,ax = plt.subplots(figsize=(8,6))
hb = ax.hexbin(glas_gdf_aea.geometry.x, glas_gdf_aea.geometry.y, C=glas_gdf_aea['glas_z'], \
               reduce_C_function=np.median, gridsize=nbins, cmap='inferno', alpha=0.6)
plt.colorbar(hb, ax=ax, label='Elevation (m HAE)')
states_gdf.to_crs(proj_str_aea).plot(ax=ax, facecolor='none', edgecolor='black');
ax.set_xlim(bbox[[0,2]]);
ax.set_ylim(bbox[[1,3]]);
ax.set_title('GLAS Bin Median Elevation');

In [ ]:
f,ax = plt.subplots(figsize=(8,6))
hb = ax.hexbin(glas_gdf_aea.geometry.x, glas_gdf_aea.geometry.y, C=glas_gdf_aea['glas_srtm_dhdt'], \
               reduce_C_function=np.median, gridsize=nbins, cmap='RdBu', alpha=0.6, vmin=-3, vmax=3)
plt.colorbar(hb, ax=ax, label='Elevation Change (m/yr)')
states_gdf.to_crs(proj_str_aea).plot(ax=ax, facecolor='none', edgecolor='black');
ax.set_xlim(bbox[[0,2]]);
ax.set_ylim(bbox[[1,3]]);
ax.set_title('Median Elevation Change (m/yr), GLAS - SRTM');

## Merge GLAS points with RGI polygons
Let's see if we can answer the following question:

*Can we identify CONUS glacier surface elevation change that occurred between SRTM (2000) and GLAS (2003-2009) data collection?*

Earlier we computed some statistics for the full CONUS GLAS sample and hex bins.  Now let's analyze the GLAS points that intersect each RGI glacier polygon. 

One approach would be to loop through each glacier polygon, and do an intersection operation with all points.  But this is inefficient, and doesn't scale well.  It is much more efficient to do a spatial join between the points and the polygons, then groupby and aggregate to compute the relevant statistics for all points that intersect each glacier polygon.

You may have learned how to perform a join or spatial join in a GIS course.  So, do we need to open ArcMap or QGIS here?  Do we need a full-fledged spatial database like PostGIS?  No!  GeoPandas has you covered.

* Start by reviewing the Spatial Join documentation here: http://geopandas.org/mergingdata.html
* Use the geopandas `sjoin` method: http://geopandas.org/reference/geopandas.sjoin.html

## First, we need to make sure all inputs have the same projection
* Reproject the RGI polygons to match our point CRS (custom Albers Equal-area)

In [ ]:
rgi_gdf_conus_aea = rgi_gdf_conus.to_crs(glas_gdf_aea.crs)
states_gdf_aea = states_gdf.to_crs(glas_gdf_aea.crs)

### Optional: isolate relevant columns to simplify our output

In [ ]:
glas_gdf_aea.columns

In [ ]:
rgi_gdf_conus_aea.columns

In [ ]:
glas_col = ['glas_z', 'glas_srtm_dhdt', 'geometry']
rgi_col = ['RGIId', 'Area', 'Name', 'geometry']

glas_gdf_aea_lite = glas_gdf_aea[glas_col]
rgi_gdf_conus_aea_lite = rgi_gdf_conus_aea[rgi_col]

In [ ]:
glas_gdf_aea_lite

## Now try a spatial join between these two 
* Use the GLAS points as the "left" GeoDataFrame and the RGI polygons as the "right" GeoDataFrame
* Start by using default options (`op='intersects', how='inner'`)
* Note the output geometry type and columns

In [ ]:
glas_gdf_aea_rgi = gpd.sjoin(glas_gdf_aea_lite, rgi_gdf_conus_aea_lite)
glas_gdf_aea_rgi

## Check number of records

In [ ]:
print("Number of RGI polygons before:", rgi_gdf_conus_aea.shape[0])
print("Number of GLAS points before:", glas_gdf_aea.shape[0])
print("Number of GLAS points that intersect RGI polygons:", glas_gdf_aea_rgi.shape[0])

## Check number of GLAS points per RGI polygon

In [ ]:
glas_gdf_aea_rgi['RGIId'].value_counts()

## Which glacier has the greatest number of points?

Some notes on indexing and selecting from Pandas DataFrame: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/

Here we'll use the `iloc` function to pull out the record for the RGIId key with the highest point count.

In [ ]:
label = glas_gdf_aea_rgi['RGIId'].value_counts().index[0]
print(label)

In [ ]:
rgi_maxcount = rgi_gdf_conus_aea[rgi_gdf_conus_aea['RGIId'] == label].iloc[0]
rgi_maxcount

In [ ]:
rgi_maxcount.geometry

## ipyleaflet plot

OK, great, but where is this glacier?  Let's plot on an interactive ipyleaflet map.

Note that leaflet uses tiled basemaps: https://en.wikipedia.org/wiki/Tiled_web_map 

Default projection is Web Mercator (EPSG:3857): https://en.wikipedia.org/wiki/Web_Mercator_projection.  This works well for lower latitudes, but not the polar regions.

FYI, `folium` provides similar functionality outside of the iPython/Jupyter stack: https://python-visualization.github.io/folium/

In [ ]:
from ipyleaflet import Map, Marker, basemaps

In [ ]:
#Look at all of the basemap options!
basemaps.keys()

In [ ]:
center = (rgi_maxcount['CenLat'], rgi_maxcount['CenLon'])
basemap = basemaps.Stamen.Terrain
m = Map(center=center, zoom=12, basemap=basemap)
#label=rgi_gdf_conus_aea.loc[label]['Name']
marker = Marker(location=center, draggable=True)
m.add_layer(marker);
display(m)

## Plot points and RGI polygons, Zoom in on WA state

In [ ]:
wa_state = states_gdf_aea.loc[states_gdf_aea['NAME'] == 'Washington']
#wa_geom = wa_state.iloc[0].geometry

In [ ]:
wa_bbox = wa_state.total_bounds

In [ ]:
clim = (-3.0, 3.0)
ax=states_gdf_aea.plot(facecolor='none', edgecolor='black', linewidth=0.5);
rgi_gdf_conus_aea.plot(ax=ax, edgecolor='k', lw=0.5, alpha=0.1);
glas_gdf_aea.plot(ax=ax, column='glas_srtm_dhdt', cmap='RdBu', markersize=1, legend=True, vmin=clim[0], vmax=clim[1])
ax.set_xlim(wa_bbox[[0,2]]);
ax.set_ylim(wa_bbox[[1,3]]);

## Groupby and Aggregate

OK, so we know that our sampling isn't great and our dh/dt values are noisy.  But we're here to learn some core concepts and tools, so let's compute some statistics for each glacier anyway.  Hopefully you'll see the value of these operations, and be able to reproduce in the future.

We can use the Pandas Groupby functionality to group GLAS points for each RGI polygon, and then aggregate using different functions (e.g., mean, std) for different attributes (e.g., 'glas_z', 'glas_srtm_dhdt').

This concept can feel a bit abstract at first, but it is very powerful.  

https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

In [ ]:
glas_gdf_aea_rgi.head()

In [ ]:
glas_gdf_aea_rgi.groupby('RGIId')

Hmmm.  Nothing happened.  Ah, we need a function to perform the aggregation over the grouped data!  How about taking the mean?

In [ ]:
glas_gdf_aea_rgi.groupby('RGIId').mean().head()

## Define a more sophisticated aggregation function

A dictionary of fields and functions can be used to compute a set of summary statistics for relevant columns

In [ ]:
agg_func = {'Name':'first',
            'Area':'first', 
            'glas_z':['mean', 'median', 'std', nmad],
            'glas_srtm_dhdt':['count','mean', 'median', 'std', nmad]}

In [ ]:
glas_gdf_aea_rgi_agg = glas_gdf_aea_rgi.groupby('RGIId').agg(agg_func)

In [ ]:
glas_gdf_aea_rgi_agg.head()

In [ ]:
# We don't need the multi-index here
glas_gdf_aea_rgi_agg.columns = ['_'.join(col).rstrip('_') for col in glas_gdf_aea_rgi_agg.columns.values]

In [ ]:
glas_gdf_aea_rgi_agg.head()

In [ ]:
glas_gdf_aea_rgi_agg['glas_pt_density'] = glas_gdf_aea_rgi_agg['glas_srtm_dhdt_count']/glas_gdf_aea_rgi_agg['Area_first']

In [ ]:
glas_gdf_aea_rgi_agg.plot();

## Wait a minute, what happened to our RGI polygon geometry?

This was a casualty of our initial spatial join, as we preserved the Point geometry for each GLAS record, not the RGI geometry.

Let's create a new GeoDataFrame, adding the original RGI geometry to the aggregated statistics.

Since both DataFrames have the same Index (RGIId), Pandas will automatically join with corresponding records.

In [ ]:
rgi_gdf_conus_aea['geometry'].head()

In [ ]:
#A bit of a hack to join on RGIId 
glas_gdf_aea_rgi_agg_gdf = gpd.GeoDataFrame(glas_gdf_aea_rgi_agg, geometry=rgi_gdf_conus_aea[['RGIId','geometry']].set_index('RGIId')['geometry'])

In [ ]:
glas_gdf_aea_rgi_agg_gdf.head()

# Let's make some final maps

## Import some other useful mapping packages

`matplotlib-scalebar` adds a dynamic scalebar to matplotlib axes

`contextily` downloads and statically renders basemap tiles

In [ ]:
from matplotlib_scalebar.scalebar import ScaleBar
import contextily as ctx

In [ ]:
def add_basemap(ax, crs, zoom='auto'):
    ctx.add_basemap(ax=ax, crs=crs, url=ctx.sources.ST_TERRAIN, zoom=zoom)
    #Create a scalebar object, with scaling factor of 1.0 px, since we're using projected coordinate system with unit 1 m
    scalebar = ScaleBar(1.0)
    #Add scalebar to axes
    ax.add_artist(scalebar)

In [ ]:
f, ax = plt.subplots()
glas_gdf_aea.plot(ax=ax, column='glas_z', cmap='inferno', markersize=2, legend=True)
#Add basemap, specifying crs keyword
add_basemap(ax, crs=glas_gdf_aea.crs)

In [ ]:
clim = (-5.0, 5.0)
f, axa = plt.subplots(1,2, figsize=(10,4), sharex=True, sharey=True)

rgi_gdf_conus_aea.plot(ax=axa[0], edgecolor='k', facecolor='w', lw=0.5, alpha=0.2);
glas_gdf_aea_rgi_agg_gdf.plot(ax=axa[0],column='glas_srtm_dhdt_count', cmap='inferno', edgecolor='k', lw=0.5, legend=True)
axa[0].set_title('Number of GLAS shots per RGI Polygon')
add_basemap(axa[0], glas_gdf_aea.crs)

glas_gdf_aea_rgi_agg_gdf.plot(ax=axa[1],column='glas_srtm_dhdt_median', cmap='RdBu', edgecolor='k', lw=0.5, vmin=clim[0], vmax=clim[1], legend=True)
axa[1].set_title('Median SRTM to GLAS dh/dt (m/yr)')
glas_gdf_aea_rgi.plot(ax=axa[1],column='glas_srtm_dhdt', cmap='RdBu', markersize=5, edgecolor='0.5', lw=0.5, vmin=clim[0], vmax=clim[1])
states_gdf_aea.plot(ax=axa[1], edgecolor='k', facecolor='none', lw=0.5)

plt.tight_layout()

## Zoom to North Cascades

# Can we extract any additional insight from the reduced data?

## Point count vs. Glacier Area

*Do we see more valid GLAS points over bigger glaciers?*

In [ ]:
f, ax = plt.subplots()
ax.scatter(glas_gdf_aea_rgi_agg_gdf['Area_first'], glas_gdf_aea_rgi_agg_gdf['glas_srtm_dhdt_count'])
ax.set_ylabel('Number of GLAS shots')
ax.set_xlabel('Glacier Area (km^2)');

## Median dh/dt vs. Glacier Area

Do we see less elevation change over bigger glaciers?

In [ ]:
f, ax = plt.subplots()
ax.scatter(glas_gdf_aea_rgi_agg_gdf['Area_first'], glas_gdf_aea_rgi_agg_gdf['glas_srtm_dhdt_median'])
ax.set_xlabel('Glacier Area (km^2)')
ax.set_ylabel('Median SRTM - GLAS dh/dt (m/yr)')
ax.axhline(0, color='k');

## Median dh/dt vs. Glacier Elevation

*Do we see greater elevation loss at lower elevations?*

In [ ]:
f, ax = plt.subplots()
ax.scatter(glas_gdf_aea_rgi_agg_gdf['glas_z_median'], glas_gdf_aea_rgi_agg_gdf['glas_srtm_dhdt_median'])
ax.set_xlabel('Median Glacier Elevation (m HAE)')
ax.set_ylabel('Median SRTM - GLAS dh/dt (m/yr)')
ax.axhline(0, color='k');

## Conclusion:

N/A

# Save the final polygons to a GIS-ready file

The workflows in these Notebooks are intended to be fully reproducible, starting with raw data and producing all final output.  But sometimes you want to write out geospatial data for analysis in a GUI-based GIS (QGIS, ArcMap), or to share with colleagues who will use these tools.

## Check available output formats for geopandas
* Use fiona to get a list of available file type drivers for output
* Note: the 'r' means fiona/geopandas can read this file type, 'w' means it can write this file type, 'a' means it can append to an existing file.
    * https://fiona.readthedocs.io/en/latest/manual.html#writing-vector-data

In [ ]:
import fiona
fiona.supported_drivers

## How to choose a format?
* If you've taken a GIS class (or not), you've probably used shapefiles in the past.  Please stop.  The ESRI shapefile is a legacy format, though it is still widely used.
* http://switchfromshapefile.org/
* Better options these days are Geopackage (GPKG) when spatial index is required, and simple GeoJSON for other cases
    * Both should be supported by your GIS (including QGIS, ArcGIS, etc)
* Let's use geopackage for this exercise
* Can use the Geopandas `to_file()` method to create this file
    * Make sure you properly specify filename with extension and the `driver` option
    * *Note: Writing out may take a minute, and may produce an intermediate '.gpkg-journal' file*
        * Can see this in the file browser or terminal!

In [ ]:
out_fn='./conus_glas_gdf_aea_rgi_agg.gpkg'
glas_gdf_aea_rgi_agg_gdf.to_file(out_fn, driver='GPKG')

#out_fn='./conus_glas_gdf_aea_rgi_agg.geojson'
#glas_gdf_aea_rgi_agg_gdf.to_file(out_fn, driver='GeoJSON')

In [ ]:
ls -lh $out_fn

## 🎉

You can now directly load this gpkg file in any GIS, without defining a coordinate system. You can also load this file directly into geopandas in the future using the `read_file()` method, without having to repeat the processing above.

### See for yourself!
Try it! Right-click on file in the file browser to the left of the JupyterLab interface, then select Download and pick a location on your local computer (e.g., your Downloads folder). 

Then open this file in QGIS or ArcGIS on your local machine!

# So, what if we actually had a decent GLAS sample of CONUS glaciers?

## Estimate glacier mass balance

We could estimate mass balance for each glacier polygon using the mean dh/dt, glacier area, and a standard bulk density (e.g., 850 kg/m3 [Huss, 2013]).  Could then use geopandas to perform a secondary aggregation to compile statistics for polygons representing mountain ranges, river basins, etc.

With a sparse sample, probably best to derive dh/dt vs. elevation curves, then combine with observed glacier hypsometry to estimate mass balance for glacier polygons.  This can work, but need to be careful about spatial variability.

# Final thoughts

GLAS was not the right tool for small, low-latitude CONUS glaciers.  We kind of knew this would be the case before we started, but hey, it was worth a look, and we learned some basic Python geospatial analysis skills.

The concepts and approaches presented here can be applied to larger glaciers or ice caps, especially at higher latitudes.  One could modify to repeat for all RGI regions.

We started with an existing csv of culled points here.  One could repeat with a similarly processed subset of ATL06 points using the workflows presented earlier this week.  This will provide a longer time period to evaluate noisy elevation measurements.  Replacing void-filled SRTM with another reference DEM is also needed (e.g., the timestamped USGS NED).

Note that the core tools presented here have Dask integration (https://dask.org/) to allow you to chunk and process larger-than-memory datasets with minimal modification to the code.

## Other visualization packages for large point datasets
* Datashader
* ipyvolume
* hvplot

# Appendix A: Sampling a raster at points

This is something that is surprisingly common, but may not be simple to implement.  Let's discuss a few options:
1. Simple `rasterio` sampling with integer indices using nearest neighbor
2. Statistics extracted for a circular window around each point location
3. NumPy/SciPy interpolation routines

For this example, we will use a sample 90-m SRTM-GL1 DEM over WA state, but you could repeat with any DEM (e.g., ArcticDEM)

Also, here is an `icepyx` notebook that includes an example of raster sampling: https://github.com/icesat2py/icepyx/blob/master/doc/examples/ICESat-2_DEM_comparison_Colombia_working.ipynb

## 1. Rasterio sampling

In [ ]:
import os
import rasterio as rio
from rasterio import plot

### Open the file with rasterio

https://rasterio.readthedocs.io/en/stable/

In [ ]:
srtm_fn = '/srv/shared/SRTM3_wa_mosaic_utm.tif'

In [ ]:
#This is rasterio Dataset object
srtm_src = rio.open(srtm_fn)

In [ ]:
srtm_src.profile

In [ ]:
srtm_src.crs

In [ ]:
srtm_extent = rio.plot.plotting_extent(srtm_src)

### Read as a NumPy Masked array

In [ ]:
srtm = srtm_src.read(1, masked=True)

In [ ]:
srtm

### Create a quick plot

In [ ]:
f, ax = plt.subplots()
ax.imshow(srtm, extent=srtm_extent);

## Generate shaded relief map
* Many ways to do this, but we'll just use the `gdaldem` command line utility for simplicity

In [ ]:
hs_fn = os.path.splitext(srtm_fn)[0]+'_hs.tif'
if not os.path.exists(hs_fn):
    !gdaldem hillshade $srtm_fn $hs_fn

In [ ]:
srtm_hs_src = rio.open(hs_fn)
hs = srtm_hs_src.read(1, masked=True)

## Plot color shaded relief map

In [ ]:
f, ax = plt.subplots()
ax.imshow(hs, cmap='gray', extent=rio.plot.plotting_extent(srtm_hs_src))
ax.imshow(srtm, extent=srtm_extent, alpha=0.5);

### Reproject GLAS points to match raster

In [ ]:
glas_gdf_srtm = glas_gdf_aea.to_crs(srtm_src.crs)

### Prepare the coordinate arrays to pass to rio `sample` function
* The `sample` function expects a list of (x,y) tuples: https://rasterio.readthedocs.io/en/latest/api/rasterio.sample.html
    * Need to create this from the `geometry` objects in your GeoDataFrame
    * You want a list of the form [(x1,y1),(x2,y2),…]
* Pass to `sample`
* Note that the `sample` function returns a `generator` object, and it doesn't actually evaluate the call!
* Can wrap this in a `np.array(list())` to evaluate, or use `np.fromiter()`
* This operation may take ~10-20 seconds to complete

In [ ]:
glas_coord = [(pt.x, pt.y) for pt in glas_gdf_srtm.geometry]
#glas_coord = np.vstack((glas_gdf_srtm.geometry.x.values, glas_gdf_srtm.geometry.y.values)).T

### Sample with rasterio

In [ ]:
glas_srtm_sample = srtm_src.sample(glas_coord)
glas_srtm_sample

### This is a generator, so we actually need to evaluate

In [ ]:
glas_srtm_elev = np.fromiter(glas_srtm_sample, dtype=srtm.dtype)
glas_srtm_elev

### Deal with nodata
* Some of our GLAS points are located over areas where we don't have valid DEM pixels
* These will be assigned the raster nodata value (-32768 in this case)

In [ ]:
glas_srtm_elev_ma = np.ma.masked_equal(glas_srtm_elev, srtm_src.nodata)
glas_srtm_elev_ma

### Add new column to the GeoDataFrame
* Set masked values to `np.nan` (which requires a conversion to float)

In [ ]:
glas_gdf_srtm['srtm_90m_z_rio'] = glas_srtm_elev_ma.astype(float).filled(np.nan)

In [ ]:
glas_gdf_srtm.dropna().head()

In [ ]:
f, ax = plt.subplots()
ax.imshow(hs, cmap='gray', extent=rio.plot.plotting_extent(srtm_hs_src))
#ax.imshow(srtm, extent=srtm_extent, alpha=0.5);
glas_gdf_srtm.dropna().plot('srtm_90m_z_rio', ax=ax, markersize=1);

*Note: the SRTM elevation values are height above the EGM96 geoid*

### Notes on sampling coarse rasters or noisy rasters at integer pixel locations
* The rasterio approach is efficient, but it uses a nearest neighbor algorithm to extract the elevation value for the grid cell that contains the point, regardless of where the point falls within the grid cell (center vs. corner)
* But the DEM grid cells can be big (~90x90 m for the SRTM3 data), so if point is near the corner of a pixel on steep slope, the pixel value might not be representative.
* A better approach is to use bilinear or bicubic sampling, to interpolate the elevation value at the point coordinate using pixel values within some neighborhood around the point, (e.g. 2x2 window for bilinear, 4x4 window for cubic)
* Other approaches involve computing zonal stats within some radius of the point location (e.g., median elevation of pixels within 300 m of the point, using buffer to create polygons)
    * https://www.earthdatascience.org/courses/earth-analytics-python/lidar-remote-sensing-uncertainty/extract-data-from-raster/
    * https://pysal.org/scipy2019-intermediate-gds/deterministic/gds2-rasters.html#getting-values-at-cells
    * https://github.com/dshean/pygeotools/blob/master/pygeotools/lib/geolib.py#L1019

## 2. Local window sample

https://github.com/dshean/demcoreg/blob/master/demcoreg/sample_raster_at_pts.py

https://github.com/dshean/pygeotools/blob/master/pygeotools/lib/geolib.py#L1019

## 3. Scipy ndimage: n-order polynomial
* Good option for regular grids (i.e., DEMs)
* Propagates nan, issues when DEM has missing data

In [ ]:
import scipy.ndimage
#Should dropna here
yx = np.array([glas_gdf_srtm.geometry.y, glas_gdf_srtm.geometry.x])
#Convert map coordinates to array coordinates (want float, not whole integer indices)
#See functions here
#https://rasterio.readthedocs.io/en/latest/api/rasterio.transform.html
#Need to revisit
#Use bilinear resampling here (order=1)
ndimage_samp = scipy.ndimage.map_coordinates(srtm, yx, order=1, mode='nearest')
ndimage_samp = np.ma.masked_equal(ndimage_samp, srtm_src.nodata)

In [ ]:
srtm_src.transform

In [ ]:
glas_gdf_srtm['srtm_90m_z_ndimage'] = ndimage_samp.astype(float).filled(np.nan)

In [ ]:
glas_gdf_srtm.dropna().head()

# Appendix B: Distortion in Polar Stereographic Projections

"The [polar stereographic] projection is defined on the entire sphere, except at one point: the projection point. Where it is defined, the mapping is smooth and bijective. It is conformal, meaning that it preserves angles at which curves meet. It is neither isometric nor area-preserving: that is, it preserves neither distances nor the areas of figures." [Wikipedia]

Many of you are probably using polar stereographic projections, likely the standard EPSG:3031 or EPSG:3413 projections for Antarctica and Greenland, respectively.  These are designed to minimize distortion near the latitude of true scale (71°S for Antarctica and 70°N for Greenland).  This means that area and distance distortion will increase as you move away from this latitude.  So areas or distances measured near the pole will not be equivalent to areas or distances measured on the Antarctic Peninsula.  The difference isn't huge, but is nontrivial (~1-2% for Antarctica I believe).  See https://nsidc.org/data/polar-stereo/ps_grids.html for more details.

The figures here might help illustrate: https://en.wikipedia.org/wiki/Stereographic_projection#Properties

Let's try to illustrate this nuanced, but often overlooked issue.

## Tissot's Indicatrix for polar stereographic projection

Let's use the classic Tissot's Indicatrix plots to show map distortion.

https://en.wikipedia.org/wiki/Tissot%27s_indicatrix

In [ ]:
import cartopy
import cartopy.crs as ccrs

#Cartopy implementation of EPSG:3031 and EPSG:3413
crs_3031 = ccrs.SouthPolarStereo(true_scale_latitude=-71)
crs_3413 = ccrs.NorthPolarStereo(central_longitude=-45, true_scale_latitude=70)

#Circle locations
lons = range(-180, 180, 30)
lats = range(-90, 91, 10)
#Radius of circles
rad = 400

In [ ]:
fig = plt.figure(figsize=(10,5))
ax1 = plt.subplot(1, 2, 1, projection=crs_3413)
ax1.coastlines()
ax1.gridlines(ylocs=[70,],color='r')
ax1.tissot(facecolor='orange', edgecolor='0.5', alpha=0.4, rad_km=rad, lons=lons, lats=lats)
ax1.set_title('EPSG:3413')

ax2 = plt.subplot(1, 2, 2, projection=crs_3031)
ax2.coastlines()
ax2.gridlines(ylocs=[-71,],color='r')
ax2.tissot(facecolor='orange', edgecolor='0.5', alpha=0.4, rad_km=rad, lons=lons, lats=lats)
ax2.set_title('EPSG:3031')
ax2.set_extent([-180, 180, -90, -50], ccrs.PlateCarree())

#There is a bug in cartopy extent when central_longitude is not 0
#Get extent in projected crs (x0, x1, y0, y1)
extent_3413 = ax2.get_extent()
ax1.set_extent(extent_3413, crs_3413)
plt.tight_layout()

## OK, cool plot bro.  But why does this matter?

Note the size of the circles in the corners and over the pole, relative to the circles near the latitude of true scale (red line).  While it is unlikely that you'll use this projection to look at mid-latitude regions, you can see the difference in area distortion over the ~20° of latitude between North and South Greenland.

Say you generated amazing elevation difference grids for all of the Greenland ice sheet using ICESat and ICESat-2 crossovers, and you used the EPSG:3413 projection.  Say the grids have a posting (grid cell size) of 120 m.  Imagine didividing the above plots into grid cells.  You'll end up with more grid cells over the circles at lower latitudes (e.g., South Greenland) and fewer grid cells over circles at high latitudes (e.g., North Greenland).  

Let's put it another way.  You could try to compute volume change of the Greenland ice sheet by summing dh/dt values for all grid cells in each catchment.  Let's assume all grid cells have the same -1.0 m/yr value.  The integrated estimates for the catchments in South Greenland will have more grid cells due to the projection, resulting in larger apparent negative volume change!

## Let's compare with an equal-area projection
Lambert Azimuthal Equal-Area is not a bad choice for the polar regions

In [ ]:
#Cartopy implementation of EPSG:3031 and EPSG:3413
#crs_slaea = ccrs.LambertAzimuthalEqualArea(central_longitude=0.0, central_latitude=-90.0)
crs_nlaea = ccrs.LambertAzimuthalEqualArea(central_longitude=-45.0, central_latitude=90.0)

#Specify locations of Tissot's Indicatrix
lons = range(-180, 180, 30)
lats = range(-90, 91, 10)
rad = 400

In [ ]:
#Add a 100 km grid
dx = 100000
xgrid = np.arange(extent_3413[0], extent_3413[1]+dx, dx)
ygrid = np.arange(extent_3413[2], extent_3413[3]+dx, dx)

In [ ]:
fig = plt.figure(figsize=(10,5))
ax1 = plt.subplot(1, 2, 1, projection=crs_3413)
ax1.coastlines()
ax1.tissot(facecolor='orange', edgecolor='0.5', alpha=0.4, rad_km=rad, lons=lons, lats=lats)
ax1.set_title('EPSG:3413')
#ax1.gridlines(crs=ccrs.PlateCarree(), ylocs=lats)
ax1.gridlines(crs=crs_3413, xlocs=xgrid, ylocs=ygrid)
ax1.gridlines(ylocs=[70,],color='r')

ax2 = plt.subplot(1, 2, 2, projection=crs_nlaea)
ax2.coastlines()
ax2.tissot(facecolor='orange', edgecolor='0.5', alpha=0.4, rad_km=rad, lons=lons, lats=lats)
ax2.set_title('Lambert Azimuthal Equal-Area')
ax2.set_extent([-180, 180, 50, 90], ccrs.PlateCarree())
#ax2.gridlines(crs=ccrs.PlateCarree(), ylocs=lats)
ax2.gridlines(crs=crs_nlaea, xlocs=xgrid, ylocs=ygrid)

#ax1.set_extent(extent_3413, crs_3413)
#ax2.set_extent(extent_3413, crs_3413)

gr_extent = [-1.4E6,1.4E6,-3.7E6,-5.3E5]
ax1.set_extent(gr_extent, crs_3413)
ax2.set_extent(gr_extent, crs_3413)

Not the distorted shape of the circles near Southern Greenland, though they should have identical area.

## Bonus: the "Flat Earth" Projection: Azimuthal Equidistant

![Azimuthal Equidistant](https://upload.wikimedia.org/wikipedia/commons/2/2f/Flat_earth.png)

>An azimuthal equidistant projection of the entire spherical Earth. A rendered picture of the Flat Earth model. The white around the outside of the globe is thought to be an 'Ice Wall', preventing people from falling off the surface of the earth. [Wikipedia]

So, those of you who have been to the South Pole, can you help me understand how this works?